# Introduction

In [1]:
%run "./Getting started.ipynb"
import requests
base_url = 'http://localhost:5000'
while True:
    try:
        r=requests.get(base_url)
        if r.text == 'EMERGENT API':
            break
    except:
        continue
print(r.text)

/Users/robbiefasano/emergent/emergent
Overwriting /Users/robbiefasano/emergent/emergent/networks/test/things/test_thing.py
Overwriting /Users/robbiefasano/emergent/emergent/networks/test/hubs/test_hub.py
Overwriting /Users/robbiefasano/emergent/emergent/networks/test/network.py
DataDict([('thing', {'X': None, 'Y': None})])
New state: DataDict([('thing', {'X': 1, 'Y': 2})])
 * Serving Flask app "emergent.API.API" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
EMERGENT API


In order to start a process, we POST a settings dictionary to the /run endpoint containing all of the info necessary to define the process. All of the following is handled much simpler by the GUI, but this tutorial demonstrates the exact steps required to manually launch an optimization through the API. Let's construct a command for a grid search optimization of the "gaussian" experiment attached to our hub:

In [2]:
hub = network.hubs['hub']
hub.range['thing'] = {'X': {'min': -2, 'max': 2}, 'Y': {'min': -2, 'max': 2}}
thing = hub.children['thing']
state = {'thing': thing.state}
settings = {'hub': hub.name, 'state': state}

settings['experiment'] = {'name': 'gaussian',
                          'params': {'sigma_x': 0.3, 
                                     'sigma_y': 0.8, 
                                     'x0': 0.3, 
                                     'y0': 0.6, 
                                     'noise':0} }
settings['sampler'] = {'name': 'Grid',
                       'params': {'Steps': 20, 'Sweeps': 1} }

settings['process'] = {'type': 'model', 'end at': 'Best point', 'callback': None}

Now let's submit this process via the API:

In [ ]:
import requests
requests.post('http://127.0.0.1:5000/run', json=settings)

In [ ]:
import requests
samplers = requests.get('http://127.0.0.1:5000/hubs/hub/samplers').json()
requests.get('http://127.0.0.1:5000/hubs/hub/samplers/%s/plot/data'%samplers[0])

Instead of calling an optimization through the API, let's construct it explicitly:

In [ ]:
from emergent.API.blueprints.rpc import prepare_sampler
sampler = prepare_sampler(settings, network)
sampler._solve()

It's also possible to chain multiple optimizations together:

In [ ]:
from copy import deepcopy
from emergent.API.blueprints.rpc import prepare_sampler

coarse_settings = deepcopy(settings)
coarse_settings['sampler']['params']['Steps']=5
coarse_sampler = prepare_sampler(coarse_settings, network)

fine_settings = deepcopy(settings)
fine_settings['sampler']['params']['Steps']=30
fine_sampler = prepare_sampler(fine_settings, network)

points, costs = coarse_sampler._solve()
points2, costs2 = fine_sampler._solve()


We could define a class to help us manage these successive processes:

In [17]:
import numpy as np

class Pipeline:
    def __init__(self):
        self._points = []
        self._costs = []
        self.points = np.empty((0,2))
        self.costs = []
        self.processes = []
        self.models = []
        
    def add_process(self, process):
        self.processes.append(process)
        
    def add_model(self, model):
        self.models.append(model)
        
    def run_all(self):
        for process in self.processes:
            points, costs = process()
            self._points.append(points)
            self._costs.append(costs)
            self.points = np.append(self.points, points, axis=0)
            self.costs = np.append(self.costs, costs)
            
        for model in self.models:
            model.fit(self.points, self.costs)
      
    
from copy import deepcopy
from emergent.API.blueprints.rpc import prepare_sampler


settings['sampler']['params']['Steps']=5
coarse_sampler = prepare_sampler(settings, network)

settings['sampler']['params']['Steps']=30
fine_sampler = prepare_sampler(settings, network)

from emergent.models.nonlinear import Nonlinear
model = Nonlinear()
model.prepare(fine_sampler)

p = Pipeline()
p.add_process(coarse_sampler._solve)
p.add_model(model)
p.add_process(fine_sampler._solve)

p.run_all()

INFO:root:Optimization complete!
INFO:root:Optimization complete!


In [18]:
model.predict([[0,0]])

(array([-7.6895334e-31]), 0)

array([0, 1, 0, 2, 0, 1, 0, 2])

array([], shape=(0, 2), dtype=float64)